In [1]:
# importing Required Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)# To see all the columns of a dataframe
#pd.set_option('display.max_rows', None)

In [2]:
# Function to reduce the memory usage of various Dataframes
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
       
        1. Iterate over every column
        2. Determine if the column is numeric
        3. Determine if the column can be represented by an integer
        4. Find the min and the max value
        5. Determine and apply the smallest datatype that can fit the range of values

    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [3]:
def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
# Loading reduced feature training set
X_train = import_data('X_train_final.csv')
y_train = pd.read_csv('y_train.final.csv')

Memory usage of dataframe is 181.24 MB
Memory usage after optimization is: 38.27 MB
Decreased by 78.9%


In [5]:
# Loading reduced feature test set
X_test = import_data('X_test_final.csv')
y_test = pd.read_csv('y_test.final.csv')

Memory usage of dataframe is 60.41 MB
Memory usage after optimization is: 12.76 MB
Decreased by 78.9%


## Loading all the best models from various categories

In [6]:
# Importing Joblib module
import joblib

In [7]:
# Importing best Logistic regression Classifier
lr = joblib.load('Log_Reg.joblib')

In [8]:
# Importing best Random Forest Classifier
rf = joblib.load('Rand_Forest_optuna_reg_2.joblib')

In [9]:
# Importing best Light Gbm Classifier
lgbm = joblib.load('Light_Gbm_2.joblib')

In [10]:
# Importing best Linear Discriminant Analysis Classifier
lda = joblib.load('Vanilla_lda.joblib')

## Model_9: Voting Classifier with Default Weights & Soft_Voting.

In [11]:
# Importing Voting classifier from sklearn
from sklearn.ensemble import VotingClassifier

In [12]:
# Importing the Sklearn's roc_auc_score module
from sklearn.metrics import roc_auc_score

In [13]:
# Instantiating the voting Classifier Object
voting_clf = VotingClassifier(estimators=[('logistic_Reg',lr),('Random_Forest',rf),('Light_Gbm',lgbm),
                                           ('Linear_Dis',lda)],voting='soft',n_jobs=5

In [14]:
# Fitting the default voting classifier on the Reduced Feature Training set
voting_clf.fit(X_train.values, y_train)

VotingClassifier(estimators=[('logistic_Reg',
                              LogisticRegression(C=0.0031160262723627184,
                                                 class_weight={0: 1.0, 1: 14},
                                                 l1_ratio=0.4046164083668398,
                                                 n_jobs=5, penalty='elasticnet',
                                                 random_state=42,
                                                 solver='saga')),
                             ('Random_Forest',
                              RandomForestClassifier(class_weight={0: 1.0,
                                                                   1: 13},
                                                     max_depth=18,
                                                     min_samples_leaf=0.0010028580411287713,
                                                     n_estimators=570, n_jobs=5,
                                                     ra...
             

In [17]:
#Defining the function to calculate the roc_auc score for the feature sets
def cal_roc_auc(X, y, cls, f_set, t_set, model_name):
    ''' Calculates the roc auc score using the best study parameters 
        f_set : String: specifies 'full feature', 'Reduced feature'
        t_set: String: specifies 'training', 'test'
        model_name: String: specifies Name of the model '''
        
    y_pred = cls.predict_proba(X)
    print('The roc_auc_score for the {} {} set using the {} is '.format(f_set,t_set,model_name),roc_auc_score(y,y_pred[:,1]))

In [18]:
# Calculating the reduced feature training set roc_auc score using the best study parameters
cal_roc_auc(X_train.values, y_train, voting_clf,'reduced feature','training',' default Voting Classifier')

The roc_auc_score for the reduced feature training set using the  default Voting Classifier is  0.8046460997742609


In [19]:
# Calculating the reduced feature test roc_auc score using the best study parameters
cal_roc_auc(X_test.values, y_test, voting_clf,'reduced feature','test','default Voting Classifier')

The roc_auc_score for the reduced feature test set using the default Voting Classifier is  0.7664216167852927


In [20]:
# Saving the default voting Classifier
import joblib
joblib.dump(voting_clf,'Voting_default.joblib')

['Voting_default.joblib']

### Calculating R_R ratio for default Voting Classifier.

In [21]:
# Importing required Libraries
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [22]:
# Instantiating the Stratified K fold object
cv_strat = StratifiedKFold(5,random_state=42)

In [23]:
# Computing the CV scores using sklearn's cross_val_score
score_voting_default = cross_val_score(voting_clf, X_train.values, y_train, cv=cv_strat, n_jobs=5, scoring='roc_auc')

In [24]:
print('The reward associated with the default Voting Classifier using roc_auc metric is: ',np.mean(score_voting_default))

The reward associated with the default Voting Classifier using roc_auc metric is:  0.7634440208535519


In [25]:
print('The risk associated with the default Voting Classifier using roc_auc metric is: ',np.std(score_voting_default))

The risk associated with the default Voting Classifier using roc_auc metric is:  0.004740966801500991


In [26]:
R_R_Ratio_voting_default = np.mean(score_voting_default)/np.std(score_voting_default)

In [27]:
print('The reward risk ratio for the default Voting Classifier using roc_auc metric is: ',R_R_Ratio_voting_default)

The reward risk ratio for the default Voting Classifier using roc_auc metric is:  161.03129442118123


In [28]:
# Saving the default voting Classifier
import joblib
joblib.dump(voting_clf,'Voting_default.joblib')

['Voting_default.joblib']

#### R_R Ratio for the default Voting classifier using roc_auc metric is: 161.03129442118123

## Observation(s): 
### 1 The default voting classifier slightly overfits the dataset. May be hyperparameter tuning give better results.
### 2) The test set roc_auc score for the default voting classifier is more than that of component Logistic Regression , LDA & Random Forest models, but less than that of  component Light GBM. May be tuning the weights of voting classifier result in better results.
### 3) On the other hand, the R_R ratio for the default voting classifier model is more than those of component Logistic regression &  LDA models, but less than those of component Random forest & LightGBM models. So clearly LightGBM bests default Voting classifier hands down.

## Model_10: Voting Classifier with Tuned weights using Optuna.

In [33]:
# Importing  hyperparamater tuning optimizer optuna
import optuna

In [29]:
# Instantiating a new voting classifier object
voting_clf_2 = VotingClassifier(estimators=[('logistic_Reg',lr),('Random_Forest',rf),('Light_Gbm',lgbm),
                                           ('Linear_Dis',lda)],voting='soft',n_jobs=5)

In [30]:
# Defining the appropriate objective function for the best weights of Voting classifier.
def objective_wrappper_Vt_2(X_tr, y_tr, cls=None, cv_strat=None):
    '''
    Optimizes voting classifier (cls) weights parameter on the given training set X_tr,y_tr
    using Startified Cross Validation object cv_strat
      
    '''
    
    def objective(trial):
        w1 = trial.suggest_uniform('w1',-1,1)
        w2 = trial.suggest_uniform('w2',-1,1)
        w3 = trial.suggest_uniform('w3',-1,1)
        w4 = trial.suggest_uniform('w4',-1,1)
        
        params = {
            'weights':[w1,w2,w3,w4]                 
            }
        
        cls.set_params(**params)#Initializing the model with the parameter
               
        return np.mean(cross_val_score(cls, X_tr, y_tr, cv=cv_strat, n_jobs=5, scoring='roc_auc'))
    
    return objective

#### Computing the best weights for the Voting Classifier_2 using Optuna.

In [31]:
# Defining the evaluation function for study's best parameters
def study_best_score_params(X_tr, y_tr, cls, obj_func, cv_strat, n_trials=50):
    ''' Computes the best hyper parameters of the classsifier and returns 
    Optuna's study's best score & clasifier parameters'''
    study = optuna.create_study(direction='maximize')
    study.optimize(obj_func(X_tr, y_tr, cls, cv_strat), n_trials)
    best_score = study.best_value
    best_params = study.best_params
    return (best_score,best_params)


In [34]:
# Extracting the best model parameters and best study score for the Voting Classifier_2
best_study_score,best_study_params = study_best_score_params(X_train.values, y_train, voting_clf_2, objective_wrappper_Vt_2,
                                                   cv_strat)

[I 2021-01-02 22:42:11,653] A new study created in memory with name: no-name-fddb9447-6269-434e-99b7-abd2c26e3f32
[I 2021-01-02 22:52:12,920] Trial 0 finished with value: 0.7584916705328142 and parameters: {'w1': 0.2909599590279115, 'w2': -0.6691209098646473, 'w3': -0.7251220701550498, 'w4': 0.2916882657943256}. Best is trial 0 with value: 0.7584916705328142.
[I 2021-01-02 22:58:28,783] Trial 1 finished with value: 0.7412433492892578 and parameters: {'w1': -0.853989929814851, 'w2': -0.13971544751084086, 'w3': 0.19613224534500429, 'w4': 0.2336487277016317}. Best is trial 0 with value: 0.7584916705328142.
[I 2021-01-02 23:08:05,313] Trial 2 finished with value: 0.40354532412477895 and parameters: {'w1': 0.8045767323509665, 'w2': -0.25248762510719014, 'w3': -0.3556876468814054, 'w4': -0.25565137159829177}. Best is trial 0 with value: 0.7584916705328142.
[I 2021-01-02 23:17:54,578] Trial 3 finished with value: 0.7629465414379971 and parameters: {'w1': 0.9979255001393559, 'w2': 0.6308369696

[I 2021-01-03 03:38:41,163] Trial 31 finished with value: 0.767345617517084 and parameters: {'w1': 0.18649111208932578, 'w2': 0.1518919329166348, 'w3': -0.8287162773015736, 'w4': -0.9011449604707698}. Best is trial 23 with value: 0.767866739648184.
[I 2021-01-03 03:47:58,589] Trial 32 finished with value: 0.7677674844146847 and parameters: {'w1': 0.12953540839802258, 'w2': -0.0803412748728825, 'w3': -0.9541124181143825, 'w4': -0.7991500924254487}. Best is trial 23 with value: 0.767866739648184.
[I 2021-01-03 03:57:27,587] Trial 33 finished with value: 0.7679089500995767 and parameters: {'w1': -0.03264723028188459, 'w2': -0.029930046315313815, 'w3': -0.9817864974187314, 'w4': -0.7289013094360486}. Best is trial 33 with value: 0.7679089500995767.
[I 2021-01-03 04:06:10,324] Trial 34 finished with value: 0.7674653910936595 and parameters: {'w1': -0.037984138971208725, 'w2': -0.029894363655682064, 'w3': -0.4048777834508386, 'w4': -0.3697355529003958}. Best is trial 33 with value: 0.7679089

In [35]:
print('The best roc_auc_score for the study is: ',best_study_score)

The best roc_auc_score for the study is:  0.7681115084496509


In [36]:
print('The best study parameters for the classifier are: ',best_study_params)

The best study parameters for the classifier are:  {'w1': 0.03698328444546678, 'w2': 0.06481993464516658, 'w3': -0.5684888767601572, 'w4': -0.4152300140118141}


In [37]:
# Obtaining the best tuned Voting clasiifier model by setting best study parameters.
voting_clf_2 = voting_clf_2.set_params(weights=[best_study_params['w1'],best_study_params['w2'],
                                                 best_study_params['w3'],best_study_params['w4']])

In [45]:
# fitting the best Voting Classifier on the reduced feature training set
voting_clf_2.fit(X_train.values, y_train)

VotingClassifier(estimators=[('logistic_Reg',
                              LogisticRegression(C=0.0031160262723627184,
                                                 class_weight={0: 1.0, 1: 14},
                                                 l1_ratio=0.4046164083668398,
                                                 n_jobs=5, penalty='elasticnet',
                                                 random_state=42,
                                                 solver='saga')),
                             ('Random_Forest',
                              RandomForestClassifier(class_weight={0: 1.0,
                                                                   1: 13},
                                                     max_depth=18,
                                                     min_samples_leaf=0.0010028580411287713,
                                                     n_estimators=570, n_jobs=5,
                                                     ra...
             

In [46]:
# Calculating the reduced feature training set roc_auc score using the best study parameters
cal_roc_auc(X_train.values, y_train, voting_clf_2, 'reduced feature', 'training', 'tuned Voting Classifier_2')

The roc_auc_score for the reduced feature training set using the tuned Voting Classifier_2 is  0.835313505875434


In [47]:
# Calculating the reduced feature test roc_auc score using the best study parameters
cal_roc_auc(X_test.values, y_test, voting_clf_2, 'reduced feature', 'test', 'tuned Voting Classifier_2')

The roc_auc_score for the reduced feature test set using the tuned Voting Classifier_2 is  0.7718169510820518


In [48]:
# Saving the tuned voting Classifier
import joblib
joblib.dump(voting_clf_2,'Voting_class_2.joblib')

['Voting_class_2.joblib']

### Calculating R_R ratio for  tuned  Voting Classifier_2.

In [49]:
# Computing the CV scores using sklearn's cross_val_score
score_voting_2 = cross_val_score(voting_clf_2, X_train.values, y_train, cv=cv_strat, n_jobs=5, scoring='roc_auc')

In [50]:
print('The reward associated with the tuned Voting Classifier using roc_auc metric is: ',np.mean(score_voting_2))

The reward associated with the tuned Voting Classifier using roc_auc metric is:  0.7681115084496509


In [51]:
print('The risk associated with the tuned Voting Classifier using roc_auc metric is: ',np.std(score_voting_2))

The risk associated with the tuned Voting Classifier using roc_auc metric is:  0.004689520879208718


In [52]:
R_R_Ratio_voting_2 = np.mean(score_voting_2)/np.std(score_voting_2)

In [54]:
print('The reward risk ratio for the tuned Voting Classifier_2 using roc_auc metric is: ',R_R_Ratio_voting_2)

The reward risk ratio for the tuned Voting Classifier_2 using roc_auc metric is:  163.79317380910297


#### R_R Ratio for the tuned Voting classifier_2 using roc_auc metric is: 163.79317380910297

## Observation(s): 
### 1 The tuned voting classifier_2  clearly overfits the dataset, even more than the default version.
### 2) The test set roc_auc score of the tuned voting classifier_2 is more than that of all component models (by a good margin ), but for that of the component Light  GBM and also exceeds that of untuned  default version.
### 3) Further, the R_R ratio for the tuned voting classifier model is more than that of the default voting classifier, but still less than those of component Light Gbm and Random Forest classifiers.

#### Lets try to further reduce overfitting by incorporating more regularization & tuning another Voting Classifier_3.

In [55]:
# Instantiating a new voting classifier object
voting_clf_3 = VotingClassifier(estimators=[('logistic_Reg',lr),('Random_Forest',rf),('Light_Gbm',lgbm),
                                           ('Linear_Dis',lda)],voting='soft',n_jobs=5)

In [56]:
# Defining the appropriate objective function for the best positive weights of Voting classifier.
def objective_wrappper_Vt_3(X_tr, y_tr, cls=None, cv_strat=None):
    '''
    Optimizes voting classifier (cls) weights parameter on the given training set X_tr,y_tr
    using Startified Cross Validation object cv_strat
      
    '''
    
    def objective(trial):
        w1 = trial.suggest_uniform('w1',0,1)
        w2 = trial.suggest_uniform('w2',0,1)
        w3 = trial.suggest_uniform('w3',0,1)
        w4 = trial.suggest_uniform('w4',0,1)
        
        params = {
            'weights':[w1,w2,w3,w4]                 
            }
        
        cls.set_params(**params)#Initializing the model with the parameter
               
        return np.mean(cross_val_score(cls, X_tr, y_tr, cv=cv_strat, n_jobs=5, scoring='roc_auc'))
    
    return objective

In [57]:
# Extracting the best model parameters and best study score for the 2nd Voting Classifier
best_study_score,best_study_params = study_best_score_params(X_train.values, y_train, voting_clf_3, objective_wrappper_Vt_3,
                                                   cv_strat)

[I 2021-01-03 16:17:20,425] A new study created in memory with name: no-name-fa386478-8884-420c-a068-b82c3791a2df
[I 2021-01-03 16:26:52,137] Trial 0 finished with value: 0.767001674343009 and parameters: {'w1': 0.10774709169418406, 'w2': 0.38599189026976755, 'w3': 0.8853934833368124, 'w4': 0.38023088614583844}. Best is trial 0 with value: 0.767001674343009.
[I 2021-01-03 16:33:18,847] Trial 1 finished with value: 0.7651768753272742 and parameters: {'w1': 0.6030994300522659, 'w2': 0.007367845048253496, 'w3': 0.6054506354708481, 'w4': 0.29620703388471237}. Best is trial 0 with value: 0.767001674343009.
[I 2021-01-03 16:43:06,373] Trial 2 finished with value: 0.7629358580486899 and parameters: {'w1': 0.44802670997137584, 'w2': 0.6872856666660636, 'w3': 0.47999145703522017, 'w4': 0.41705356716325026}. Best is trial 0 with value: 0.767001674343009.
[I 2021-01-03 16:52:27,956] Trial 3 finished with value: 0.7662861409995485 and parameters: {'w1': 0.1204096485118118, 'w2': 0.2789102775748597

[I 2021-01-03 21:14:31,673] Trial 31 finished with value: 0.7680473701172844 and parameters: {'w1': 0.16673348359527287, 'w2': 0.2048822718166385, 'w3': 0.9936841271245903, 'w4': 0.08078247461755628}. Best is trial 24 with value: 0.7681490341404353.
[I 2021-01-03 21:24:01,149] Trial 32 finished with value: 0.768412265682939 and parameters: {'w1': 0.006035602899010558, 'w2': 0.04583130695137036, 'w3': 0.9898415019476203, 'w4': 0.15451161504144328}. Best is trial 32 with value: 0.768412265682939.
[I 2021-01-03 21:33:21,203] Trial 33 finished with value: 0.7684261743094537 and parameters: {'w1': 0.01862824610080053, 'w2': 0.03973089077337316, 'w3': 0.827427548199444, 'w4': 0.15565261694130678}. Best is trial 33 with value: 0.7684261743094537.
[I 2021-01-03 21:42:08,486] Trial 34 finished with value: 0.7684413153265106 and parameters: {'w1': 0.10395391465678885, 'w2': 0.04090546684700298, 'w3': 0.8452422937130442, 'w4': 0.13864802359714418}. Best is trial 34 with value: 0.7684413153265106.

In [58]:
print('The best roc_auc_score for the study is: ',best_study_score)

The best roc_auc_score for the study is:  0.7685422339897621


In [59]:
print(('The best study parameters for the classifier are: ',best_study_params))

('The best study parameters for the classifier are: ', {'w1': 0.07036755001796159, 'w2': 0.006986420679320118, 'w3': 0.9494789443218892, 'w4': 0.143312740080169})


In [60]:
# Obtaining the best tuned voting clasiifier model by setting best study parameters.
voting_clf_3 = voting_clf_3.set_params(weights=[best_study_params['w1'],best_study_params['w2'],
                                                 best_study_params['w3'],best_study_params['w4']])

In [61]:
# Fitting the best tuned Voting Classifier_3 on the reduced feature training set.
voting_clf_3.fit(X_train.values, y_train)

VotingClassifier(estimators=[('logistic_Reg',
                              LogisticRegression(C=0.0031160262723627184,
                                                 class_weight={0: 1.0, 1: 14},
                                                 l1_ratio=0.4046164083668398,
                                                 n_jobs=5, penalty='elasticnet',
                                                 random_state=42,
                                                 solver='saga')),
                             ('Random_Forest',
                              RandomForestClassifier(class_weight={0: 1.0,
                                                                   1: 13},
                                                     max_depth=18,
                                                     min_samples_leaf=0.0010028580411287713,
                                                     n_estimators=570, n_jobs=5,
                                                     ra...
             

In [62]:
# Calculating the reduced feature training set roc_auc score using the best study parameters
cal_roc_auc(X_train.values, y_train, voting_clf_3, 'reduced feature', 'training', 'tuned Voting Classifier_3')

The roc_auc_score for the reduced feature training set using the tuned Voting Classifier_3 is  0.8376093835249532


In [64]:
# Calculating the reduced feature test set roc_auc score using the best study parameters
cal_roc_auc(X_test.values, y_test, voting_clf_3, 'reduced feature', 'test', 'tuned Voting Classifier_3')

The roc_auc_score for the reduced feature test set using the tuned Voting Classifier_3 is  0.7721661743865859


In [65]:
# Saving the best voting Classifier_3
import joblib
joblib.dump(voting_clf_3,'Voting_class_3.joblib')

['Voting_class_3.joblib']

### Calculating R_R ratio for the tuned Voting Classifier_3.

In [66]:
# Computing the CV scores using sklearn's cross_val_score
score_voting_3 = cross_val_score(voting_clf_3, X_train.values, y_train, cv=cv_strat, n_jobs=5, scoring='roc_auc')

In [67]:
print('The reward associated with the tuned Voting Classifier_3 using roc_auc metric is: ',np.mean(score_voting_3))

The reward associated with the tuned Voting Classifier_3 using roc_auc metric is:  0.7685422339897621


In [69]:
print('The risk associated with the tuned Voting Classifier_3 using roc_auc metric is: ',np.std(score_voting_3))

The risk associated with the tuned Voting Classifier_3 using roc_auc metric is:  0.004654394145759493


In [70]:
R_R_Ratio_voting_3 = np.mean(score_voting_3)/np.std(score_voting_3)

In [72]:
print('The reward risk ratio for the tuned Voting Classifier_3 using roc_auc metric is: ',R_R_Ratio_voting_3)

The reward risk ratio for the tuned Voting Classifier_3 using roc_auc metric is:  165.12186332349236


In [74]:
print('5 fold CV roc_auc scores for the tuned Voting Classifier_3 are: ',score_voting_3)

5 fold CV roc_auc scores for the tuned Voting Classifier_3 are:  [0.76345684 0.76346698 0.77567379 0.7708476  0.76926596]


#### R_R Ratio for the tuned Voting classifier_3 using roc_auc metric is: 165.12186332349236

## Observation(s): 
### 1 The tuned voting classifier_3  clearly overfits the dataset, even slightly more than the voting classifier_2.
### 2) The test set roc_auc score of the tuned voting classifier_3 is again more than that of all component models (by a good margin ), but for that of the component Light  GBM and also exceeds those of both untuned  default version & Voting classifier_2
### 3) Further, the R_R ratio for the tuned voting classifier_3 model is more than those of both voting classifier_2 & the default version , but still less than that of the component Random Forest classifier.

### Best R_R Ratio for the voting classifier family using roc_auc metric is:  165.12186332349236 ,corresponding to tuned Voting Classifier_3.

### Taking everything into consideration, the tuned Voting Classifier_3 , even with slight overfitting, dominates all other voting classifiers & is clearly the best in the family of Voting Classifiers for this dataset.